In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("../data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

2025/10/16 20:28:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/16 20:28:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/10/16 20:28:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 20:28:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:28:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run finalizado. RMSE = 8.9926


# Model Registry example

In [3]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
import pickle
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

<Experiment: artifact_location='file:///c:/ITESOO/Datos_proyecto/nyc-taxi-predictions-2025/notebooks/mlruns/2', creation_time=1760498454348, experiment_id='2', last_update_time=1760498454348, lifecycle_stage='active', name='nyc-taxi-model-registry-example', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
mlflow.sklearn.autolog()

In [5]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [6]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [7]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            #!mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/16 20:29:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/16 20:29:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\ITESOO\Datos_proyecto\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/16 20:30:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


### Registrar modelo

In [13]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/14 21:50:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:50:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/14 21:50:42 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/10/14 21:50:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/14 21:50:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/14 21:50:47 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/14 21:50:47 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INF

### Metodo 2 para registrar modelos

In [8]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

2025/10/16 20:31:57 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/16 20:31:57 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/16 20:31:57 WARNING mlflow.tracking._model_registry.fluent: Run with id 2ad8e655434e4be49f04b10c0936aae3 has no artifacts at artifact path 'model', registering model based on models:/m-b62c1c26767e4f74a7342b7f2e4351d9 instead
Created version '4' of model 'nyc-taxi-model'.


### Tercer metodo

In [13]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [16]:
# client.create_registered_model(name="nyc-taxi-model")

In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

### Asignar alias

In [ ]:
from mlflow import MlflowClient

# client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [20]:
# create "champion" alias for version 1 of model "example-model"
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=1
)

In [21]:
# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=5
)

In [22]:
# get a model version by alias
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1760499560564, current_stage='None', deployment_job_state=None, description='', last_updated_timestamp=1760499560564, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='ecf31aafe0f5408a9228e2c9b88a5db3', run_link='', source='file:///c:/ITESOO/Datos_proyecto/nyc-taxi-predictions-2025/notebooks/mlruns/2/models/m-c6417e5b737841168f604a4a1a3c9e00/artifacts', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [ ]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [23]:
client.update_model_version(
    name="nyc-taxi-model",
    version=5,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=['challenger'], creation_timestamp=1760668736308, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1760669459886, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='a1fae92255304f61b5fe153b5c27beb5', run_link=None, source='runs:/a1fae92255304f61b5fe153b5c27beb5/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

### Obtener por version

In [24]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 5

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

array([15.02976639, 15.02976639, 15.02976639, ..., 15.02976639,
       15.02976639, 15.02976639], shape=(44218,))

In [26]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "challenger"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([15.02976639, 15.02976639, 15.02976639, ..., 15.02976639,
       15.02976639, 15.02976639], shape=(44218,))

### Comparar versiones

In [ ]:
# Create the directory if it doesn't exist
# !mkdir -p ./data

# Download files using curl
# !curl -o ./data/green_tripdata_2025-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-03.parquet

### Comparar modelos

In [27]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [30]:
df = read_dataframe("../data/green_tripdata_2025-03.parquet")

In [31]:
# Descargar preprocessador
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

'c:\\ITESOO\\Datos_proyecto\\nyc-taxi-predictions-2025\\notebooks\\preprocessor'

In [32]:
# import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [33]:
X_test = preprocess(df, dv)

In [34]:
target = "duration"
y_test = df[target].values

In [44]:
test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

{'rmse': 6.01147524270201}

In [45]:
test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)

{'rmse': 9.162768346025532}

In [47]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=5
)

# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)